# **Data Collection**

In this notebook, we will preprocess the raw collected datasets, and create the project's dataset.

In order to run this notebook, please create two directories:

- ```raw_datasets``` - a directory designated for the raw collected datasets.
- ```processed_datasets``` - a directory designated for saving the processed datasets.

In addition, please upload **all** raw datasets to the ```raw_datasets``` directory.

*The raw datasets can be downloaded from the project's GitHub repository using the following [link](https://github.com/dan-israeli/Elections-and-Inflation-Causal-Inference-Research/tree/main/Data/Raw%20Datasets)

In [ ]:
# imports
import pandas as pd
import datetime as dt

## **Data Preprocessing**

In this section, we will preprocess the collected raw datasets.

Let us define constansts and helper functions which will help us in the preprocessing procedure:

In [ ]:
### constants
RELEVANT_COUNTRIES = ['Austria', 'Belgium', 'Cyprus', 'Czech Republic', 'Denmark',
                      'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary',
                      'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta',
                      'Netherlands', 'Poland', 'Portugal', 'Slovakia', 'Slovenia',
                      'Spain', 'Sweden', 'United Kingdom']

RELEVANT_YEARS = [year for year in range(2004, 2020)]


### helper functions
def get_prev_election_num_years(c_year, country_election_years):

  prev_election_years = [year for year in country_election_years if year < c_year]
  lastest_prev_year = max(prev_election_years)

  prev_election_num_years = c_year - lastest_prev_year
  return prev_election_num_years


def extract_data_coutry_year_format(raw_data, col_name):

  records = []
  for country in RELEVANT_COUNTRIES:
    country_data = raw_data[raw_data["Country"] == country]

    for year in RELEVANT_YEARS:
      c_val = country_data[country_data["Year"] == year][col_name].values[0]
      prev_val = country_data[country_data["Year"] == year - 1][col_name].values[0]

      records.append((country, year, c_val, prev_val))

  col_names = ["Country", "Year", col_name, f"Previous Year {col_name}"]
  processed_data = pd.DataFrame(records, columns=col_names)

  return processed_data


def extract_data_year_format(raw_data, col_name):

  records = []
  for year in RELEVANT_YEARS:
    c_val = raw_data[raw_data["Year"] == year][col_name].values[0]
    prev_val = raw_data[raw_data["Year"] == year-1][col_name].values[0]

    records.append((year, c_val, prev_val))

  col_names = ["Year", col_name, f"Previous Year {col_name}"]
  processed_data = pd.DataFrame(records, columns=col_names)

  return processed_data


def change_countries_names(country_name):

  if country_name == "Czechia":
    return "Czech Republic"

  if country_name == "Slovak Republic":
    return "Slovakia"

  return country_name


def get_val(val, units):

  if units == "K":
    val /= 10**3

  elif units == "M":
    val /= 10**6

  return round(val, 3)


def process_data_world_bank_group(raw_data, col_name, units=None):

  # keep only relevant columns
  relevant_cols = ["Country Name"] + ['2003'] + [str(year) for year in RELEVANT_YEARS]
  raw_data = raw_data[relevant_cols]

  # change countries names
  raw_data["Country Name"] = raw_data["Country Name"].apply(change_countries_names)

  # keep only relevant countries
  raw_data = raw_data[raw_data["Country Name"].isin(RELEVANT_COUNTRIES)]

  records = []
  for country in RELEVANT_COUNTRIES:
    # get the country's records
    country_data = raw_data[raw_data["Country Name"] == country]

    # get each year value
    for year in RELEVANT_YEARS:
      c_val = get_val(country_data[str(year)].values[0], units)
      prev_val = get_val(country_data[str(year-1)].values[0], units)

      records.append((country, year, c_val, prev_val))

  # create a dataframe of the processed data
  col_names = ["Country", "Year", col_name, f"Previous Year {col_name}"]
  processed_data = pd.DataFrame(records, columns=col_names)
  return processed_data


def process_stock_data(raw_data, col_name, price_type):
  # group the data by yearly quarters
  # calculate the mean close price of each yearly quarter
  q_data = raw_data.groupby(pd.Grouper(key="Date", freq="Q"))[price_type].mean().round(3).reset_index()

  records = []
  for year in RELEVANT_YEARS:
    year_q_values = q_data[q_data["Date"].dt.year == year][price_type].values.tolist()
    prev_year_q_values = q_data[q_data["Date"].dt.year == year - 1][price_type].values.tolist()

    records.append([year] + year_q_values + prev_year_q_values)

  # create a dataframe of the processed data
  col_names = ["Year"] + \
              [f"{col_name} Q{i}" for i in range(1, 5)] + \
              [f"Previous Year {col_name} Q{i}" for i in range(1, 5)]

  processed_data = pd.DataFrame(records, columns=col_names)
  return processed_data


def calc_weighted_avg(df, col_name):
  weighted_sum, total_days = 0, 0

  for _, row in df.iterrows():
    # calculate the days difference between the dates
    days_diff = (row["End Date"] - row["Start Date"]).days

    weighted_sum += row[col_name] * days_diff
    total_days += days_diff

  weighted_avg = round(weighted_sum / total_days, 3)
  return weighted_avg


def calc_weighted_avg_f(col_name):
  return lambda df: calc_weighted_avg(df, col_name)

### **Election Year**

The election year dataset was collected from the Wikipedia website.

Let us extract 'is Election Year' (binary) and 'Years Since Last Election' variables for each year:

In [ ]:
raw_election_year = pd.read_csv("raw_datasets/raw_election_years.csv")
raw_election_year.head()

records = []
for country in RELEVANT_COUNTRIES:
  country_election_years = raw_election_year[raw_election_year["Country"] == country]["Election Year"].values.tolist()

  for year in RELEVANT_YEARS:
    is_election_year = int(year in country_election_years)
    prev_election_num_years = get_prev_election_num_years(year, country_election_years)

    records.append((country, year, is_election_year, prev_election_num_years))

col_names = ["Country", "Year", "Is Election Year", "Years Since Last Election"]
processed_election_year = pd.DataFrame(records, columns=col_names)

display(processed_election_year)

,Country,Year,Is Election Year,Years Since Last Election
0,Austria,2004,0,2
1,Austria,2005,0,3
2,Austria,2006,1,4
3,Austria,2007,0,1
4,Austria,2008,1,2
...,...,...,...,...
395,United Kingdom,2015,1,5
396,United Kingdom,2016,0,1
397,United Kingdom,2017,1,2
398,United Kingdom,2018,0,1


Now, let us save the processed dataset:

In [ ]:
processed_election_year.to_csv("processed_datasets/processed_election_years.csv", index=False)

### **Inflation Rate**

The inflation rate dataset was collected from the World Bank Group website.

Let us extract the values of the current year and the previous one for each country and year:

In [ ]:
raw_inflation_rate = pd.read_csv("raw_datasets/raw_inflation_rates.csv")
processed_inflation_rate = process_data_world_bank_group(raw_inflation_rate, col_name="Inflation Rate")

display(processed_inflation_rate)

<ipython-input-7-5a96b443b831>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data["Country Name"] = raw_data["Country Name"].apply(change_countries_names)


,Country,Year,Inflation Rate,Previous Year Inflation Rate
0,Austria,2004,2.061,1.356
1,Austria,2005,2.299,2.061
2,Austria,2006,1.442,2.299
3,Austria,2007,2.169,1.442
4,Austria,2008,3.216,2.169
...,...,...,...,...
395,United Kingdom,2015,0.368,1.451
396,United Kingdom,2016,1.008,0.368
397,United Kingdom,2017,2.558,1.008
398,United Kingdom,2018,2.293,2.558


Now, let us save the processed dataset:

In [ ]:
processed_inflation_rate.to_csv("processed_datasets/processed_inflation_rates.csv", index=False)

###**GDP Per Capita**

The GDP per capita dataset was collected from the World Bank Group website.

Let us extract the values of the current year and the previous one for each country and year:

In [ ]:
raw_gdp_per_captia = pd.read_csv("raw_datasets/raw_gdp_per_capita.csv")
processed_gdp_per_captia = process_data_world_bank_group(raw_gdp_per_captia, col_name="GDP Per Capita")

display(processed_gdp_per_captia)

<ipython-input-7-5a96b443b831>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data["Country Name"] = raw_data["Country Name"].apply(change_countries_names)


,Country,Year,GDP Per Capita,Previous Year GDP Per Capita
0,Austria,2004,36889.234,32294.049
1,Austria,2005,38417.458,36889.234
2,Austria,2006,40669.327,38417.458
3,Austria,2007,46915.337,40669.327
4,Austria,2008,51919.984,46915.337
...,...,...,...,...
395,United Kingdom,2015,44964.391,47439.617
396,United Kingdom,2016,40985.235,44964.391
397,United Kingdom,2017,40572.121,40985.235
398,United Kingdom,2018,43203.814,40572.121


Now, let us save the processed dataset:

In [ ]:
processed_gdp_per_captia.to_csv("processed_datasets/processed_gdp_per_captia.csv", index=False)

### **Currency is Euro**

The currency is Euro dataset was collected from the European Commission website.

Let us extract the values of the current year and the previous one for each country and year:

In [ ]:
raw_is_euro = pd.read_csv("raw_datasets/raw_currency_is_euro.csv")
processed_is_euro = extract_data_coutry_year_format(raw_is_euro, col_name="Currency is Euro")

display(processed_is_euro)

,Country,Year,Currency is Euro,Previous Year Currency is Euro
0,Austria,2004,1,1
1,Austria,2005,1,1
2,Austria,2006,1,1
3,Austria,2007,1,1
4,Austria,2008,1,1
...,...,...,...,...
395,United Kingdom,2015,0,0
396,United Kingdom,2016,0,0
397,United Kingdom,2017,0,0
398,United Kingdom,2018,0,0


Now, let us save the processed dataset:

In [ ]:
processed_is_euro.to_csv("processed_datasets/processed_currency_is_euro.csv", index=False)

###**Net Migration**

The net migration dataset was collected from the World Bank Group website.

Let us extract the values of the current year and the previous one for each country and year:

In [ ]:
raw_net_migration = pd.read_csv("raw_datasets/raw_net_migration.csv")
processed_net_migration = process_data_world_bank_group(raw_net_migration, col_name="Net Migration", units="K")

display(processed_net_migration)

<ipython-input-7-5a96b443b831>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data["Country Name"] = raw_data["Country Name"].apply(change_countries_names)


,Country,Year,Net Migration,Previous Year Net Migration
0,Austria,2004,54.907,44.003
1,Austria,2005,50.282,54.907
2,Austria,2006,25.420,50.282
3,Austria,2007,23.729,25.420
4,Austria,2008,25.499,23.729
...,...,...,...,...
395,United Kingdom,2015,254.218,261.389
396,United Kingdom,2016,246.940,254.218
397,United Kingdom,2017,239.014,246.940
398,United Kingdom,2018,230.577,239.014


Now, let us save the processed dataset:

In [ ]:
processed_net_migration.to_csv("processed_datasets/processed_net_migration.csv", index=False)

###**Population**

The population size dataset was collected from the World Bank Group website.

Let us extract the values of the current year and the previous one for each country and year:

In [ ]:
raw_population = pd.read_csv("raw_datasets/raw_population.csv")
processed_population = process_data_world_bank_group(raw_population, col_name="Population", units="M")

display(processed_population)

<ipython-input-7-5a96b443b831>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data["Country Name"] = raw_data["Country Name"].apply(change_countries_names)


,Country,Year,Population,Previous Year Population
0,Austria,2004,8.172,8.121
1,Austria,2005,8.228,8.172
2,Austria,2006,8.269,8.228
3,Austria,2007,8.295,8.269
4,Austria,2008,8.321,8.295
...,...,...,...,...
395,United Kingdom,2015,65.116,64.602
396,United Kingdom,2016,65.612,65.116
397,United Kingdom,2017,66.059,65.612
398,United Kingdom,2018,66.460,66.059


Now, let us save the processed dataset:

In [ ]:
processed_population.to_csv("processed_datasets/processed_population.csv", index=False)

###**Unemployment Rate**

The unemployment rate dataset was collected from the World Bank Group website.

Let us extract the values of the current year and the previous one for each country and year:

In [ ]:
raw_unemployment_rate = pd.read_csv("raw_datasets/raw_unemployment_rates.csv")
processed_unemployment_rate = process_data_world_bank_group(raw_unemployment_rate, col_name="Unemployment Rate")

display(processed_unemployment_rate)

<ipython-input-7-5a96b443b831>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data["Country Name"] = raw_data["Country Name"].apply(change_countries_names)


,Country,Year,Unemployment Rate,Previous Year Unemployment Rate
0,Austria,2004,5.969,4.785
1,Austria,2005,5.682,5.969
2,Austria,2006,5.320,5.682
3,Austria,2007,4.909,5.320
4,Austria,2008,4.198,4.909
...,...,...,...,...
395,United Kingdom,2015,5.517,6.363
396,United Kingdom,2016,4.868,5.517
397,United Kingdom,2017,4.452,4.868
398,United Kingdom,2018,4.120,4.452


Now, let us save the processed dataset:

In [ ]:
processed_unemployment_rate.to_csv("processed_datasets/processed_unemployment_rates.csv", index=False)

### **EURO600**

The Euro600 index historic price dataset was collected from the  MarketWatch website.

Let us extract the quartly values of the current year and the previous one for each year:

*Note that The quartely price is calculated as the average close price of all days associated with the quarter.

In [ ]:
raw_euro600 = pd.read_csv("raw_datasets/raw_euro600_prices.csv",
                          parse_dates=["Date"],
                          date_format="%d/%m/%Y")

processed_euro600 = process_stock_data(raw_euro600, col_name="EURO600", price_type="Close")

processed_euro600.head()

,Year,EURO600 Q1,EURO600 Q2,EURO600 Q3,EURO600 Q4,Previous Year EURO600 Q1,Previous Year EURO600 Q2,Previous Year EURO600 Q3,Previous Year EURO600 Q4
0,2004,238.027,240.424,235.231,245.879,188.130,196.806,211.703,221.874
1,2005,260.272,265.985,286.587,298.606,238.027,240.424,235.231,245.879
2,2006,324.837,324.870,328.165,355.710,260.272,265.985,286.587,298.606
3,2007,371.690,389.813,377.537,372.779,324.837,324.870,328.165,355.710
4,2008,320.708,314.825,278.678,209.527,371.690,389.813,377.537,372.779


Now, let us save the processed data:

In [ ]:
processed_euro600.to_csv("processed_datasets/processed_euro600_prices.csv", index=False)

### **S&P500**

The S&P500 index historic price dataset was collected from the Yahoo Finance website.

Let us extract the quartly values of the current year and the previous one for each year:

*The quartely price is calculated as the average close price of all days associated with the quarter.

In [ ]:
raw_sp500 = pd.read_csv("raw_datasets/raw_sp500_prices.csv",
                        parse_dates=["Date"],
                        date_format="%d/%m/%Y")

processed_sp500 = process_stock_data(raw_sp500, col_name="S&P500", price_type="Close")

processed_sp500.head()

,Year,S&P500 Q1,S&P500 Q2,S&P500 Q3,S&P500 Q4,Previous Year S&P500 Q1,Previous Year S&P500 Q2,Previous Year S&P500 Q3,Previous Year S&P500 Q4
0,2004,1132.671,1123.196,1103.908,1162.652,860.760,937.995,1000.380,1056.453
1,2005,1191.948,1181.974,1224.174,1230.468,1132.671,1123.196,1103.908,1162.652
2,2006,1283.655,1280.812,1288.317,1388.635,1191.948,1181.974,1224.174,1230.468
3,2007,1424.381,1497.176,1489.463,1495.745,1283.655,1280.812,1288.317,1388.635
4,2008,1350.661,1371.628,1252.025,911.977,1424.381,1497.176,1489.463,1495.745


Now, let us save the processed data:

In [ ]:
processed_sp500.to_csv("processed_datasets/processed_sp500_prices.csv", index=False)

### **Crude Oil**

The crude oil historic price dataset was collected from the Yahoo Finance website.

Let us extract the quarterly values of the current year and the previous one for each year:

*The quartely price is calculated as the average close price of all days associated with the quarter.

In [ ]:
raw_crude_oil = pd.read_csv("raw_datasets/raw_crude_oil_prices.csv",
                            parse_dates=["Date"],
                            date_format="%d/%m/%Y")

processed_crude_oil = process_stock_data(raw_crude_oil, col_name="Crude Oil", price_type="Close")

processed_crude_oil.head()

,Year,Crude Oil Q1,Crude Oil Q2,Crude Oil Q3,Crude Oil Q4,Previous Year Crude Oil Q1,Previous Year Crude Oil Q2,Previous Year Crude Oil Q3,Previous Year Crude Oil Q4
0,2004,35.250,38.284,43.892,48.272,33.801,28.909,30.211,31.160
1,2005,50.031,53.220,63.312,60.046,35.250,38.284,43.892,48.272
2,2006,63.478,70.724,70.502,60.174,50.031,53.220,63.312,60.046
3,2007,58.271,65.024,75.148,90.504,63.478,70.724,70.502,60.174
4,2008,97.818,123.799,118.221,59.077,58.271,65.024,75.148,90.504


Now, let us save the processed data:

In [ ]:
processed_crude_oil.to_csv("processed_datasets/processed_crude_oil_prices.csv", index=False)

### **EUR-USD Exchange Rate**

The EUR-USD historic exchange rate dataset was collected from the European Central Bank website.

Let us extract the quartly values of the current year and the previous one for each year:

*The quartely exchange rate is calculated as the average exchange rate of all days associated with the quarter.

In [ ]:
raw_eur_use_er = pd.read_csv("raw_datasets/raw_eur_usd_exchange_rates.csv",
                             parse_dates=["Date"],
                             date_format="%d/%m/%Y")

processed_eur_usd_er = process_stock_data(raw_eur_use_er, col_name="EUR-USD Exchange Rate", price_type="Exchange Rate")

processed_eur_usd_er.head()

,Year,EUR-USD Exchange Rate Q1,EUR-USD Exchange Rate Q2,EUR-USD Exchange Rate Q3,EUR-USD Exchange Rate Q4,Previous Year EUR-USD Exchange Rate Q1,Previous Year EUR-USD Exchange Rate Q2,Previous Year EUR-USD Exchange Rate Q3,Previous Year EUR-USD Exchange Rate Q4
0,2004,1.250,1.205,1.222,1.298,1.073,1.137,1.125,1.189
1,2005,1.311,1.259,1.220,1.188,1.250,1.205,1.222,1.298
2,2006,1.202,1.258,1.274,1.289,1.311,1.259,1.220,1.188
3,2007,1.311,1.348,1.374,1.449,1.202,1.258,1.274,1.289
4,2008,1.498,1.562,1.505,1.318,1.311,1.348,1.374,1.449


Now, let us save the processed data:

In [ ]:
processed_eur_usd_er.to_csv("processed_datasets/processed_eur_usd_exchange_rates.csv", index=False)

### **Value Added Tax (VAT)**

The VAT rates dataset was collected from x.

Let us extract the values of the current year and the previous one for each country and year:

* The yearly VAT rate of a country is calculated as the weighted average of all the VAT rates in that year (with the weights being the corresponding time periods).

In [ ]:
raw_vat_rates = pd.read_csv("raw_datasets/raw_vat_rates.csv",
                            parse_dates=["Start Date", "End Date"],
                            date_format="%d/%m/%Y")

# compute for each country and year combination its VAT rate as a weighted average
weighted_avg_vat_rates = raw_vat_rates.groupby(["Country", "Year"])\
                                   .apply(calc_weighted_avg_f("VAT"))\
                                   .reset_index()\
                                   .rename(columns={0: "VAT"})

processed_vat_rates = extract_data_coutry_year_format(weighted_avg_vat_rates, col_name="VAT")

display(processed_vat_rates)

,Country,Year,VAT,Previous Year VAT
0,Austria,2004,20.0,20.0
1,Austria,2005,20.0,20.0
2,Austria,2006,20.0,20.0
3,Austria,2007,20.0,20.0
4,Austria,2008,20.0,20.0
...,...,...,...,...
395,United Kingdom,2015,20.0,20.0
396,United Kingdom,2016,20.0,20.0
397,United Kingdom,2017,20.0,20.0
398,United Kingdom,2018,20.0,20.0


Now, let us save the processed data:

In [ ]:
processed_vat_rates.to_csv("processed_datasets/processed_vat_rates.csv")

### **European Central Bank Interest Rate**

The European Central Bank (ECB) interest rate dataset was collected from European Central Bank website.

Let us extract the values of the current year and the previous one for each year:

* The yearly interest rate is calculated as the weighted average of all the interest rates in that year (with the weights being the corresponding time periods).

In [ ]:
raw_ecb_interest_rates = pd.read_csv("raw_datasets/raw_ecb_interest_rates.csv",
                                     parse_dates=["Start Date", "End Date"],
                                     date_format="%d/%m/%Y")

weighted_avg_ecb_interest_rates = raw_ecb_interest_rates.groupby("Year")\
                                                        .apply(calc_weighted_avg_f("ECB Interest Rate"))\
                                                        .reset_index()\
                                                        .rename(columns={0: "ECB Interest Rate"})

processed_ecb_interest_rates = extract_data_year_format(weighted_avg_ecb_interest_rates,
                                                        col_name="ECB Interest Rate")

processed_ecb_interest_rates.head()

,Year,ECB Interest Rate,Previous Year ECB Interest Rate
0,2004,2.000,2.259
1,2005,2.017,2.000
2,2006,2.758,2.017
3,2007,3.839,2.758
4,2008,3.904,3.839


Now, let us save the processed data:

In [ ]:
processed_ecb_interest_rates.to_csv("processed_datasets/proccessed_ecb_interest_rates.csv", index=False)

## **Project's Dataset Creation**

In this section, we will create the project's dataset by joining all  processed datasets (created in the prevoius section).

Note that we can devide the different variables into two main categories:

- Local - apply to a combination of a country and a year.
- Global - apply to all countries in a certain year.

As a result:

- For the local varaibles, we will join them based on country and year.
- For the global variables, we will join them based on year only.

Let us create the project's dataset:

In [ ]:
# define the local datasets
local_datasets = [processed_election_year, processed_inflation_rate, processed_gdp_per_captia,
                  processed_is_euro, processed_net_migration, processed_population,
                  processed_unemployment_rate, processed_vat_rates]

# define the global datasets
global_datasets = [processed_euro600, processed_sp500, processed_crude_oil,
                   processed_eur_usd_er, processed_ecb_interest_rates]

final_dataset = local_datasets[0].copy()

# join the local datasets based on the 'Country' and 'Year' colunms
for local_dataset in local_datasets[1:]:
  final_dataset = final_dataset.merge(local_dataset, on=["Country", "Year"])

# join the global datasets based on the 'Year' column
for global_dataset in global_datasets:
  final_dataset = final_dataset.merge(global_dataset, on=["Year"])

fina_dataset = final_dataset.sort_values(by=["Country", "Year"])

final_dataset.head()

,Country,Year,Is Election Year,Years Since Last Election,Inflation Rate,Previous Year Inflation Rate,GDP Per Capita,Previous Year GDP Per Capita,Currency is Euro,Previous Year Currency is Euro,...,EUR-USD Exchange Rate Q1,EUR-USD Exchange Rate Q2,EUR-USD Exchange Rate Q3,EUR-USD Exchange Rate Q4,Previous Year EUR-USD Exchange Rate Q1,Previous Year EUR-USD Exchange Rate Q2,Previous Year EUR-USD Exchange Rate Q3,Previous Year EUR-USD Exchange Rate Q4,ECB Interest Rate,Previous Year ECB Interest Rate
0,Austria,2004,0,2,2.061,1.356,36889.234,32294.049,1,1,...,1.25,1.205,1.222,1.298,1.073,1.137,1.125,1.189,2.0,2.259
1,Belgium,2004,0,1,2.097,1.589,35429.408,30655.209,1,1,...,1.25,1.205,1.222,1.298,1.073,1.137,1.125,1.189,2.0,2.259
2,Cyprus,2004,0,3,2.286,4.139,23792.621,20252.238,0,0,...,1.25,1.205,1.222,1.298,1.073,1.137,1.125,1.189,2.0,2.259
3,Czech Republic,2004,0,2,2.760,0.119,11749.853,9818.568,0,0,...,1.25,1.205,1.222,1.298,1.073,1.137,1.125,1.189,2.0,2.259
4,Denmark,2004,0,3,1.154,2.075,46511.598,40458.777,0,0,...,1.25,1.205,1.222,1.298,1.073,1.137,1.125,1.189,2.0,2.259


Now, let us save the final result:

In [ ]:
final_dataset.to_csv("dataset.csv", index=False)